## 作業目標: 使用Jieba進行各項的斷詞操作

這份作業我們會使用中文飯店評論資料集來作為斷詞練習。 [資料集:ChnSentiCorp_htl_all](https://github.com/SophonPlus/ChineseNlpCorpus)

### 讀入文本資料

In [17]:
import pandas as pd

# hint: 可利用pandas讀取CSV
###<your code>###

pd_corpus = pd.read_csv('ChnSentiCorp_htl_all.csv')
pd_corpus.head(5)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [3]:
#確認所有留言,正評價(label=1)與負評價數量(label=0)
###<your code>###
pd_positive = pd_corpus[pd_corpus.label == 1]
pd_negative = pd_corpus[pd_corpus.label == 0]

print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7766, Positive: 5322, Negative: 2444


In [22]:
#移除缺失值
###<your code>###

pd_corpus.dropna(inplace=True)

### 建構結巴斷詞Function

建構可將讀入的Pandas DataFrame的文本資料，外加一欄名為`cut`的review欄位斷詞結果

In [1]:
import jieba

In [26]:
class JiebaCutingClass(object):
    '''Class to use jeiba to parse corpus from dataframe and cut the corpus
    
    Parameters
    -----------
    key_to_cut: str
        the dataframe key to parse the sentence for jieba cutting
    dic: str
        the dictionary to use for jieba, default is None (use default dictionary)
    userdict: str
        the user defined dictionary to use for jieba, default is None
    '''
    
    def __init__(self, key_to_cut:str, dic:str=None, userdict:str=None):
        
        if dic is not None:
            jieba.set_dictionary(dic)
        
        if userdict is not None:
            jieba.load_userdict(userdict)
        
        self.key_to_cut = key_to_cut
        
        #將paddle 功能開啟
        ###<your code>###
        jieba.enable_paddle()
    @staticmethod
    def cut_single_sentence(sentence, use_paddle=False, use_full=False, use_search=False):
        
        if use_search:
            # hint:使用收尋引擎模式進行斷詞
            ###<your code>###
            out = jieba.lcut_for_search(sentence)
        else:
            # hint:非收尋引擎模式斷詞，請注意有精確模式、全模式與paddle模式
            ###<your code>###
            out = jieba.lcut(sentence,use_paddle=use_paddle,cut_all=use_full)
        return out
            
    
    def cut_corpus(self, corpus: pd.DataFrame, mode: str) -> pd.DataFrame:
        '''Method to read and cut sentence from dataframe and append another column named cut
        
        Paremeters
        --------------
        corpus: pd.DataFrame
            Input corpus in dataframe
        mode: str
            Jieba mode to be used
        
        Return
        ----------------
        out: pd.Dataframe
            Output corpus in dataframe
        '''
        
        # checking valid mode
        if mode not in ['paddle', 'full', 'precise', 'search']:
            raise TypeError(f'only support `paddle`, `full`, `precise`, and `search` mode, but get {mode}')
            
        # cut the corpus based on mode
        # hint: 根據mode來呼叫相對應的斷詞method
        if mode == 'paddle':
            ###<your code>###
            corpus['cut'] = corpus[self.key_to_cut].map(lambda x:jieba.lcut(x,use_paddle=True))
        elif mode == 'full':
            ###<your code>###
            corpus['cut'] = corpus[self.key_to_cut].map(lambda x:jieba.lcut(x,cut_all=True))
        elif mode == 'precise':
            ###<your code>###
            corpus['cut'] = corpus[self.key_to_cut].map(lambda x:jieba.lcut(x))
        elif mode == 'search':
            ###<your code>###
            corpus['cut'] = corpus[self.key_to_cut].map(lambda x:jieba.lcut_for_research(x))
        return corpus
    
    #def _paddle_cut(self, corpus):
    #    '''paddle mode
    #    '''
    #    #enable paddle
    #    #hint:先啟用paddle mode
    #    ### <your code> ###
    #    
    #    out = []
    #    # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
    #    for single_review in corpus[self.key_to_cut]:
    #        ###<your code>###
    #    
    #    corpus['cut'] = out
    #    
    #    return corpus
    
    #def _full_cut(self, corpus):
    #    '''full mode
    #    '''
    #    
    #    out = []
    #    # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
    #    for single_review in corpus[self.key_to_cut]:
    #        ###<your code>###
    #    
    #    corpus['cut'] = out
    #    
    #    return corpus
    #
    #def _precise_cut(self, corpus):
    #    '''precise mode
    #    '''
    #    
    #    out = []
    #    # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
    #    for single_review in corpus[self.key_to_cut]:
    #        ###<your code>###
    #    
    #    corpus['cut'] = out
    #    
    #    return corpus
    #
    #def _search_cut(self, corpus):
    #    '''search mode
    #    '''
    #    
    #    out = []
    #    # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
    #    for single_review in corpus[self.key_to_cut]:
    #        ###<your code>###
    #    
    #    corpus['cut'] = out
    #    
    #    return corpus

### 使用建構好的斷詞物件對文本進行斷詞

In [24]:
### 請使用精確模式與預設字典對文本進行斷詞

# hint:請先實例化JiebaCutingClass，再進行斷詞
###<your code>###
JBC = JiebaCutingClass(key_to_cut='review')

pd_cut = JBC.cut_corpus(pd_corpus,mode='full')
###<your code>### #為了避免處理時間過久, 這裡我們只使用前50個進行斷詞

pd_cut.head(50)

Paddle enabled successfully......


,label,review,cut
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙, 公路, 较, 近, ,, 但是, 公交, 指示, 不对, ,, 如果, 是..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大, 床, 房, ，, 房间, 很大, ，, 床, 有, 2M, 宽, ，, 整体..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, 的,..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小街, 街道, 上, ，, 不大, 不大好, 大好, 好找, ，, 但, 还..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD, 中心, ,, 周围, 没什么, 什么, 店铺, ,, 说, 5, 星, 有点, ..."
5,1,总的来说，这样的酒店配这样的价格还算可以，希望他赶快装修，给我的客人留些好的印象,"[总的来说, 来说, ，, 这样, 的, 酒店, 配, 这样, 的, 价格, 还算, 可以,..."
6,1,价格比比较不错的酒店。这次免费升级了，感谢前台服务员。房子还好，地毯是新的，比上次的好些。早...,"[价格, 价格比, 比比, 比较, 不错, 的, 酒店, 。, 这次, 免费, 升级, 了,..."
7,1,不错，在同等档次酒店中应该是值得推荐的！,"[不错, ，, 在, 同等, 档次, 酒店, 中, 应该, 该是, 值得, 推荐, 的, ！]"
8,1,入住丽晶，感觉很好。因为是新酒店，的确有淡淡的油漆味，房间内较新。房间大小合适，卫生间设备齐...,"[入住, 丽, 晶, ，, 感觉, 很, 好, 。, 因为, 是, 新, 酒店, ，, 的确..."
9,1,1。酒店比较新，装潢和设施还不错，只是房间有些油漆味。2。早餐还可以，只是品种不是很多。3。...,"[1, 。, 酒店, 比较, 新, ，, 装潢, 和, 设施, 还, 不错, ，, 只是, ..."


### 載入繁中字典為指定的字詞進行斷詞

In [27]:
test_string = '我愛cupoy自然語言處理馬拉松課程'
jieba_cut = JiebaCutingClass(key_to_cut='review', dic='dict.txt.big')


out_string = jieba_cut.cut_single_sentence(test_string,use_paddle=True)###<your code>### #paddle 模式
print(f'Paddle模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string,use_full=True) ###<your code>### #全模式
print(f'全模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string,use_search=True)###<your code>### #搜尋模式
print(f'搜尋模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string)###<your code>### #精確模式
print(f'精確模式: {[string for string in out_string]}')

Paddle enabled successfully......
Building prefix dict from /Users/genechen/Desktop/NLP100Days/dict.txt.big ...


Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']


Dumping model to file cache /var/folders/8s/ydlls5hj4wn0lby7ht9hrpf00000gn/T/jieba.u2f19e9c867715b1a324d8581cf01c043.cache
Loading model cost 1.891 seconds.
Prefix dict has been built successfully.


全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然', '語言', '自然語言', '處理', '馬拉', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
